In [6]:
import requests
import pandas as pd
from plyer import notification  
from datetime import datetime

def alerta(nivel, etapa):
    if nivel == 1:
        titulo_notificacao = 'Sucesso - Alerta Baixo'
    elif nivel == 2:
        titulo_notificacao = 'Erro Grave - Alerta Médio'
    elif nivel == 3:
        titulo_notificacao = 'Erro Muito Grave - Alerta Alto'
    else:
        titulo_notificacao = 'Alerta Desconhecido'

    nome_arquivo = "Script de Extração de APIs"

    data_hora_atual = datetime.now().strftime('%d-%m-%y %H:%M:%S')

    msg_notificacao = f"Base: {nome_arquivo}\nEtapa: {etapa}\n{data_hora_atual}"

    # Notificação no sistema
    notification.notify(
        title=titulo_notificacao,
        message=msg_notificacao,
        app_name='Extrator de APIs',
        timeout=10
    )


# URLs dos diferentes endpoints
urls = {
    'Tabela1': 'https://brasilapi.com.br/api/cnpj/v1/30306294000145', 
    'Tabela2': 'https://brasilapi.com.br/api/cep/v1/{CEP}', 
    'Tabela3': 'https://brasilapi.com.br/api/ibge/uf/v1/{Sigla_UF}'
}

# Dicionário para armazenar as tabelas
tabelas = {}

for nome, url in urls.items():
    # Requisição para cada tabela
    response = requests.get(url)
    
    if response.status_code == 200:
        dados_json = response.json()
        dados_list = [dados_json]
        
        # Criar DataFrame e armazenar no dicionário
        tabelas[nome] = pd.DataFrame(dados_list)
        print(f"{nome} extraída com sucesso!")
        
        # Envia alerta de sucesso (nível 1) para código 200
        alerta(1, f"{nome} extraída com sucesso.")
        
        # Extração do CEP e da Sigla_UF de Tabela1 para usar em Tabela2 e Tabela3
        if nome == 'Tabela1':
            CEP = dados_json['cep']  # Extrai o CEP de Tabela1
            Sigla_UF = dados_json['uf']  # Extrai o estado (sigla UF) de Tabela1
            print(f"CEP extraído de Tabela1: {CEP}")
            print(f"Sigla do estado extraída de Tabela1: {Sigla_UF}")
            
            # Atualiza a URL de Tabela2 com o CEP extraído
            urls['Tabela2'] = urls['Tabela2'].format(CEP=CEP)
            print(f"URL de Tabela2 atualizada: {urls['Tabela2']}")
            
            # Atualiza a URL de Tabela3 com a sigla do estado extraída
            urls['Tabela3'] = urls['Tabela3'].format(Sigla_UF=Sigla_UF)
            print(f"URL de Tabela3 atualizada: {urls['Tabela3']}")
        
    elif 300 <= response.status_code < 500:
        print(f"Erro grave ao acessar {nome}: {response.status_code}")
        
        # Envia alerta de erro grave (nível 2) para código 3xx ou 4xx
        alerta(2, f"Erro grave ao acessar a {nome}: {response.status_code}")
    
    elif response.status_code >= 500:
        print(f"Erro muito grave no servidor ao acessar {nome}: {response.status_code}")
        
        # Envia alerta de erro muito grave (nível 3) para código 5xx
        alerta(3, f"Erro interno grave no servidor ao acessar a {nome}: {response.status_code}")


Tabela1 extraída com sucesso!
CEP extraído de Tabela1: 22250911
Sigla do estado extraída de Tabela1: RJ
URL de Tabela2 atualizada: https://brasilapi.com.br/api/cep/v1/22250911
URL de Tabela3 atualizada: https://brasilapi.com.br/api/ibge/uf/v1/RJ
Tabela2 extraída com sucesso!
Tabela3 extraída com sucesso!


In [7]:
tabela1 = tabelas['Tabela1']
print(tabela1.head())

   uf       cep                                                qsa  \
0  RJ  22250911  [{'pais': None, 'nome_socio': 'RENATO MONTEIRO...   

             cnpj  pais email   porte    bairro numero      ddd_fax  ...  \
0  30306294000145  None        DEMAIS  BOTAFOGO  00501  02125149600  ...   

  codigo_natureza_juridica data_exclusao_do_simples  \
0                     2046                     None   

   motivo_situacao_cadastral ente_federativo_responsavel  \
0                          0                               

  identificador_matriz_filial  qualificacao_do_responsavel  \
0                           1                           10   

  descricao_situacao_cadastral descricao_tipo_de_logradouro  \
0                        ATIVA                        PRAIA   

   descricao_motivo_situacao_cadastral descricao_identificador_matriz_filial  
0                           SEM MOTIVO                                MATRIZ  

[1 rows x 48 columns]


In [8]:
tabela2 = tabelas['Tabela2']
print(tabela2.head())

        cep state            city neighborhood              street   service
0  22250911    RJ  Rio de Janeiro     Botafogo  Praia Botafogo 501  open-cep


In [9]:
tabela3 = tabelas['Tabela3']
print(tabela3)

   id sigla            nome                                       regiao
0  33    RJ  Rio de Janeiro  {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}
